In [1]:
#test the model class on JSWT simulations 
import numpy as np
import pandas as pd
#import splat
#import popsims
from astropy.coordinates import SkyCoord, Galactic
#from popsims.galaxy import Pointing, volume_calc, create_pop
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
#sns.set_style("darkgrid", {"axes.facecolor": ".95"})

#plt.style.use('fivethirtyeight')
#plt.style.use('dark_background')

from  matplotlib.colors import Normalize
import astropy.units as u
import popsims
import matplotlib
from tqdm import tqdm
from tqdm import tqdm
import matplotlib as mpl 
mpl.rcParams['axes.labelsize'] = 20
mpl.rcParams['xtick.labelsize'] =20
mpl.rcParams['ytick.labelsize'] = 20
mpl.rcParams['legend.fontsize'] = 20
mpl.rcParams['figure.titlesize'] = 20

from popsims.simulator import Population, pop_mags_from_type, get_velocities
from popsims.galaxy import Disk

In [2]:
ls= np.random.uniform(-180, 180, 1000)
bs= np.random.uniform(-90, 90, 1000)
s= SkyCoord(l=ls*u.degree, b=bs*u.degree, frame='galactic')

In [3]:
disk= Disk()
pop=Population(age_range=[0.01, 8])

In [4]:
#draw samples and assign them distances from a random direction
samples= pop.simulate(1e5)

/users/caganze/research/popsims/popsims/tools.py:463: RuntimeWarning: Mean of empty slice
  res=np.nanmean(res, axis=0)

/users/caganze/research/popsims/popsims/tools.py:467: RuntimeWarning: Mean of empty slice
  return np.nanmean(res, axis=0), np.nanstd(res, axis=0)

/Users/caganze/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,



In [5]:
#draw distances
ds= [disk.sample_distances(sx.l.radian, sx.b.radian, 0.1, 1000, len(samples), dsteps=1000) for sx in tqdm(s)]

100%|███████████████████████████████████████| 1000/1000 [01:50<00:00,  9.02it/s]


In [6]:
samples['distance']=np.random.choice(np.array(ds).flatten(), len(samples))

In [7]:
#add magnitudes
mags= pop_mags_from_type(samples.spt.values, d=samples['distance'], keys=['NIRISS_F115W'], object_type='dwarfs', reference=None)
for c in mags.columns:
    samples[c]= mags[c]

In [8]:
#add kinematics 
vels= get_velocities(samples.age.values, population='thin_disk')
for c in vels.columns:
    samples[c]= vels[c]